In [4]:
import SimFunctions
import SimClasses
import SimRNG
from datetime import datetime
from sklearn.linear_model import LinearRegression
import googlemaps
import pandas as pd
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.physics import Physics
from pyvis.network import Network
from IPython.display import IFrame
from IPython.display import HTML
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
import statsmodels.api as sm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


In [2]:
# BSD = pd.read_csv("BikeShare Data\Bike share ridership 2022-08.csv")
# readme = pd.read_excel(r"BikeShare Data\bikeshare-ridership-readme.xlsx")

# # move Trip_Duration column to position 1
# BSD["Trip_Duration"] = BSD.pop("Trip  Duration") * (1/60)  # Convert to Minutes
# BSD.insert(1, "Trip_Duration", BSD.pop("Trip_Duration"))

# np.random.seed(7)
# # create a new DataFrame with just the start and end station names
# stations = pd.DataFrame(BSD[['Start Station Name', 'End Station Name']])
# stations = stations.sample(int(0.10*(len(BSD))))
# # create a new DataFrame with the counts of trips between each pair of stations
# counts = stations.groupby(['Start Station Name', 'End Station Name']).size().reset_index(name="NumOfTrips").sort_values(by='NumOfTrips', ascending=False)
# # select the top 50 rows
# sample_size = counts.sample(7000)#int(0.05*(len(BSD))))
# # create a DataFrame with top 50 routes
# subset_routes = pd.DataFrame(sample_size[['Start Station Name', 'End Station Name', 'NumOfTrips']])
# subset_routes["End Station Name"].value_counts().head()

# subset_df = pd.DataFrame.merge(BSD, subset_routes)
# subset_df = subset_df.drop("NumOfTrips", axis=1)

# for i, row in subset_df.iterrows():
#     trip_length = row["Trip_Duration"]
#     # creates a new DataFrame that includes only the rows where the trip duration is at least 2 minutes, but no more than 24 hours, and is not missing (NaN).
#     if trip_length < 1 or trip_length * (1/60) > 0.75 or math.isnan(trip_length):
#         subset_df = subset_df.drop(index=i)


In [3]:
# stations_new = pd.DataFrame(subset_df[['Start Station Name', 'End Station Name']])
# station_trips = stations_new.groupby(['Start Station Name', 'End Station Name']).size().reset_index(name="NumOfTrips")

# # Create a new dataframe with all unique start station names
# start_stations = pd.DataFrame({'Start Station Name': station_trips['Start Station Name'].unique()})

# # Calculate the possible destinations and probabilities for each start station
# destinations = []
# dest_dic = {}
# for start_station in start_stations['Start Station Name']:
#     trips = station_trips[station_trips['Start Station Name'] == start_station]
#     total_trips = trips['NumOfTrips'].sum()
#     probabilities = trips['NumOfTrips'] / total_trips
#     dest_dic[start_station] = dict(zip(trips['End Station Name'], probabilities))
#     destinations.append(list(zip(trips['End Station Name'], probabilities)))

# # Create a new dataframe that contains the start station names and the list of possible destinations with associated probabilities
# station_probabilities = pd.DataFrame({'Start Station Name': start_stations['Start Station Name'], 'Destinations': destinations})
# station_dictionary = pd.DataFrame({"Start": dest_dic.keys(), "End": dest_dic.values()})
# station_vs_Dest = pd.DataFrame(dest_dic)


In [ ]:
class Order_Class:
    def __init__(self):
        self.Z_items = None

    def add_order(self, z_size):
        self.Z_items = z_size

    def remove_order(self):
        self.Z_items = None

#############################################
# Functions


def SizeOfDemand():
    U = SimRNG.Uniform(0, 1, 10)
    if U > 1/2:
        U1 = SimRNG.Uniform(0, 1, 11)
        D = 2 if U1 > 1/3 else 1
    else:
        U2 = SimRNG.Uniform(0, 1, 12)
        D = 3 if U2 > 1/3 else 4
    return D


def SizeOfOrder(s, S, I):
    if I < s:
        Z = S - I
    else:
        Z = 0
    return Z


def Order(s, S, K, i):
    Z_Size = SizeOfOrder(s, S, I=Inventory_CT.Xlast)
    Z_cost = Order_Cost(Inventory_CT.Xlast + Z_Size, K, i)
    Orders_C.add_order(Inventory_CT.Xlast + Z_Size)
    SimFunctions.Schedule(Calendar, "Order_Arrival", SimRNG.Uniform(0.5, 1, 2))
    return Z_cost, Z_Size


def Demand():
    Z_items = SizeOfDemand()
    Inventory_CT.Record(Inventory_CT.Xlast - Z_items)
    SimFunctions.Schedule(Calendar, "Demand", SimRNG.Expon(0.1, 3))


def Order_Arrival(Z_size):
    if Orders_C.Z_items is not None:
        Inventory_CT.Record(Orders_C.Z_items)
        Orders_C.remove_order()


def Order_Cost(Z, K, i):
    Z_cost = K + i*Z
    return Z_cost


def Holding_cost(cost):
    I_ps = max(Inventory_CT.Xlast, 0)
    I_plus_CT.Record(I_ps)
    cost = 1 * I_ps
    return cost, I_ps


def Backlog_cost(cost):
    I_ng = max((-1 * Inventory_CT.Xlast), 0)
    I_neg_CT.Record(I_ng)
    cost = 5 * I_ng
    return cost, I_ng


def Expected_cost(cost, I_ps, I_ns):
    I_ps = 0
    I_ng = 0
    if Inventory_CT.Xlast >= 0:
        ind_cost, I_ps = Holding_cost(cost)
        I_plus_CT.Record(I_ps)
    else:
        ind_cost, I_ng = Backlog_cost(cost)
        I_neg_CT.Record(I_ng)
    cost = cost + ind_cost
    return cost, ind_cost, I_ps, I_ng

####################################################################################################################################


sample_means_poly = []
sample_var_poly = []
s = [20, 20, 20, 20, 40, 40, 40, 60, 60]
S = [40, 60, 80, 100, 60, 80, 100, 80, 100]

for i_s in range(len(S)):

    ZSimRNG = SimRNG.InitializeRNSeed()
    Calendar = SimClasses.EventCalendar()

    TheQueues = []
    TheCTStats = []
    TheDTStats = []
    TheResources = []

    Inventory_CT = SimClasses.CTStat()
    Inventory_CT.Xlast = 60

    I_plus_CT = SimClasses.CTStat()
    I_neg_CT = SimClasses.CTStat()

    Item_cost_means = []
    I_plus_CT_list_means = []
    I_neg_CT_list_means = []
    Expected_Cost_list = []

    # Simulation
    for reps in range(0, n_reps, 1):

        #############################################
        # Intances
        K = 32  # setup cost
        i_cost = 3  # incremental cost per item ordered
        T = 120  # 120 months
        I_0 = 60  # inital invistory level

        cost = 0
        Z_cost = 0
        order_cost = 0
        I_ps = 0
        I_ng = 0
        time_avgs = 0
        avg_holding_cost = 0
        avg_backlog_cost = 0
        Avg_Item_cost = 0
        Expected_Cost = 0

        s = [20, 20, 20, 20, 40, 40, 40, 60, 60]
        S = [40, 60, 80, 100, 60, 80, 100, 80, 100]

        ############################################
        Orders_C = Order_Class()

        inventory_list = []
        time_list = []
        Z_cost_list = []
        Total_cost_list = []
        inv_cum_cost_list = []
        inv_per_cost_list = []
        I_ps_list = []
        I_ng_list = []
        xlast = []
        I_CT_list = []
        I_plus_CT_list = []
        I_neg_CT_list = []

        SimFunctions.SimFunctionsInit(
            Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
        for time in range(1, T):
            SimFunctions.Schedule(Calendar, "Order", time)
        SimFunctions.Schedule(Calendar, "Demand", SimRNG.Expon(0.1, 1))

        SimFunctions.Schedule(Calendar, "EndSimulation", T)
        NextEvent = Calendar.Remove()
        if NextEvent.EventType == "Demand":
            Demand()

        SimClasses.Clock = NextEvent.EventTime

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            end_time = SimClasses.Clock
            if NextEvent.EventType == "Demand":
                Demand()
            elif NextEvent.EventType == "Order":
                Z_cost, Z_size = Order(s[i_s], S[i_s], K, i_cost)
            elif NextEvent.EventType == "Order_Arrival":
                Order_Arrival(Z_size)

            Z_cost_list.append(Z_cost)
            cost, per_cost, I_ps, I_ng = Expected_cost(cost, I_ps, I_ng)
            inventory_list.append(Inventory_CT.Xlast)
            time_list.append(SimClasses.Clock)
            Total_cost_list.append(cost+Z_cost)
            inv_cum_cost_list.append(cost)
            inv_per_cost_list.append(per_cost)
            I_ps_list.append(I_ps)
            I_ng_list.append(I_ng)
            I_CT_list.append(Inventory_CT.Mean())
            I_plus_CT_list.append(I_plus_CT.Mean())
            I_neg_CT_list.append(I_neg_CT.Mean())
            xlast.append(Inventory_CT.Xlast)

            if NextEvent.EventType == "EndSimulation":
                SimFunctions.ClearStats(TheCTStats, TheDTStats)
                Inventory_CT.Clear()
                I_plus_CT.Clear()
                I_neg_CT.Clear()

        df = pd.DataFrame({"I(t)": inventory_list,
                           "I+(t)": I_ps_list,
                           "I-(t)": I_ng_list,
                           "Time": time_list,
                           "Total Cost": Total_cost_list,
                           "Accumalated Inventory Cost": inv_cum_cost_list,
                           "Inventory Cost": inv_per_cost_list,
                           "Z items Cost": Z_cost_list,
                           "Time Avg: I(t)": I_CT_list,
                           "Time Avg: I+(t)": I_plus_CT_list,
                           "Time Avg: I-(t)": I_neg_CT_list,
                           })
        averages = []
        for i in range(1, 121):
            j = next((idx for idx, val in enumerate(
                df["Time"]) if val > i), len(df["Time"]))
            avg = df["Z items Cost"][j-1:j]
            averages.append(avg)

        Avg_Item_cost = np.mean(averages)
        time_avgs = df[["Time Avg: I+(t)", "Time Avg: I-(t)"]][-1:].values[0]

        Item_cost_means.append(Avg_Item_cost)
        I_plus_CT_list_means.append(time_avgs[0])
        I_neg_CT_list_means.append(time_avgs[1])

        avg_holding_cost = 1*time_avgs[0]
        avg_backlog_cost = 5*time_avgs[1]

        Expected_Cost = avg_holding_cost + avg_backlog_cost + Avg_Item_cost
        Expected_Cost_list.append(Expected_Cost)
    sample_var_poly.append(np.var(Expected_Cost_list, ddof=1))
    sample_means_poly.append(np.mean(Expected_Cost_list))


# Simulation with 2 Stations

In [39]:
"""
MGSC
"""

Stations = [7, 15]
Station_Capacity = [10, 20]

ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

Station_1_Resource = []
Station_2_Resource = []

Station_1 = SimClasses.Resource()
Station_1.SetUnits(Stations[0])
Station_1_Resource.append(Station_1)
Queue_1 = SimClasses.FIFOQueue()

Station_2 = SimClasses.Resource()
Station_2.SetUnits(Stations[1])
TheResources.append(Bikes)
Station_2_Resource.append(Station_2)
Queue_2 = SimClasses.FIFOQueue()


Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Bikes)

NumOfBikes = sum(Stations)
Bikes.SetUnits(NumOfBikes)

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []


Full_Error = 0
Empty_Error = 0

In [53]:
global Stations
global Station_Capacity
Stations = [7, 15]
Station_Capacity = [10, 20]

def Arrival_Rate(Station_ID):
    if Station_ID == 0:
        Arrival_Rate = 2
    elif Station_ID == 1:
        Arrival_Rate = 4
    return Arrival_Rate

def Destination(Station_ID):
    print("Destination")
    Start_Station = Station_ID
    U = SimRNG.Uniform(0, 1, 10)
    D = 1 if U > 1/2 else 0
    Dest_ID = D
    return Dest_ID

def Departure(Station_ID):
    print("Departure")
    Stations[Station_ID] -= 1
    Dest_ID = Destination(Station_ID)
    SimFunctions.Schedule(Calendar, "Departure", 15*SimRNG.Lognormal(0.1, np.sqrt(0.066), 1))
    return Dest_ID   
    

def Customer_Arrival(Station_ID, Empty_Error, Full_Error):
    print("Customer Arrival")
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()
    
    if Station_Capacity[Station_ID] <= Stations[Station_ID]:  # STATION FULL
        print("Station Full")
        Full_Error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))

    Queue.Add(Customer)
    print(Queue.NumQueue())

    if (Station.Busy < Stations[Station_ID]) and (Station.Busy >= 0): # STATION NOT EMPTY
        Station.Seize(1)
        #Dest_ID = Departure(Station_ID)
        SimFunctions.Schedule(Calendar, "EndOfService", SimRNG.Erlang(Phases, MeanST, 2))
    else:
        print(Station.Busy)
        Empty_Error = Empty_Error + 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))
    
    #return Dest_ID
    

def Bike_rider_Arrival(Station_ID, Empty_Error, Full_Error):
    print("Bike Rider Arrival")
    Station_ID = np.random.random_integers(0, 1)

    SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(MeanTBA, 1))
    Customer = SimClasses.Entity()

    if Station_Capacity[Station_ID] <= Stations[Station_ID]:  # STATION FULL
        print("Station Full")
        Full_Error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))

    Queue.Add(Customer)
    print(Queue.NumQueue())

    # STATION NOT EMPTY
    if (Station.Busy < Stations[Station_ID]) and (Station.Busy >= 0):
        Station.Seize(1)
        #Dest_ID = Departure(Station_ID)
        SimFunctions.Schedule(Calendar, "EndOfService",
                              SimRNG.Erlang(Phases, MeanST, 2))
    else:
        print(Station.Busy)
        Empty_Error = Empty_Error + 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))


def Retrial(Dest_ID, Empty_Error, Full_Error):  # retrail function
    print("Retrial")
    if Dest_ID == 0:
        Queue = Queue_1
        Station = Station_1
    else:
        Queue = Queue_2
        Station = Station_2
        
    Customer = SimClasses.Entity()
    
    if Station_Capacity[Dest_ID] <= Stations[Dest_ID]:  # STATION FULL
        Full_Error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))
    else:
        Queue.Add(Customer)
        print("Que:", Queue.NumQueue())
        # STATION NOT EMPTY
        if (Station.Busy < Stations[Dest_ID]) and (Station.Busy >= 0):
            Stations[Dest_ID] -= 1
            SimFunctions.Schedule(Calendar, "EndOfService", SimRNG.Erlang(Phases, MeanST, 2))
        else:
            print(Station.Busy)
            Empty_Error = Empty_Error + 1
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))

def EndOfService(Dest_ID):
    print("EndOfService")
    if Dest_ID == 0:
        Queue = Queue_1
        Station = Station_1
    else:
        Queue = Queue_2
        Station = Station_2
    
    DepartingCustomer = Queue.Remove()
    Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)

    if Queue.NumQueue() > 0:
        
        SimFunctions.Schedule(Calendar, "EndOfService", SimRNG.Erlang(Phases, MeanST, 2))
    else:
        Stations[Dest_ID] += 1
        Station.Free(1)

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0

    Station_ID = np.random.random_integers(0, 1)

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
    SimFunctions.Schedule(Calendar, "ClearIt", WarmUp)

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(Station_ID, Empty_Error, Full_Error)
    
    elif NextEvent.EventType == "EndOfService":
        EndOfService(Station_ID)
    
    elif NextEvent.EventType == "ClearIt":
        SimFunctions.ClearStats(TheCTStats, TheDTStats)

    for time in [8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0]: # 30 min intervals
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        
        if NextEvent.EventType == "Customer_Arrival":
            Customer_Arrival(Station_ID, Empty_Error, Full_Error)
        
        elif NextEvent.EventType == "Departure":
            Dest_ID = Departure(Station_ID)
        
        elif NextEvent.EventType == "Retrial": 
            Dest_ID = Retrial(Dest_ID, Empty_Error, Full_Error)
        
        elif NextEvent.EventType == "EndOfService":
            EndOfService(Dest_ID)
            
        elif time == 12.0:
            break

        AllWaitMean.append(Wait.Mean())
        AllQueueMean.append(Queue_1.Mean())
        AllQueueNum.append(Queue_1.NumQueue())
        AllServerMean.append(Station_1.Mean())

q5_output = {"Estimated Expected Average wait:": AllWaitMean, "Estimated Expected Average queue-length:": AllQueueMean,
                "Estimated Expected Queue Number:": AllQueueNum, "Estimated Expected Average utilization:": AllServerMean}
q5_output


Customer Arrival
1


NameError: name 'Station' is not defined

In [ ]:
"""
MGSC
"""

Stations = [7, 15]
Station_Capacity = [10, 20]

ZSimRNG = SimRNG.InitializeRNSeed()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

Station_1_Resource = []
Station_2_Resource = []

Station_1 = SimClasses.Resource()
Station_1.SetUnits(Stations[0])
Station_1_Resource.append(Station_1)
Queue_1 = SimClasses.FIFOQueue()

Station_2 = SimClasses.Resource()
Station_2.SetUnits(Stations[1])
TheResources.append(Bikes)
Station_2_Resource.append(Station_2)
Queue_2 = SimClasses.FIFOQueue()


Calendar = SimClasses.EventCalendar()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheQueues.append(Queue)
TheResources.append(Bikes)

NumOfBikes = sum(Stations)
Bikes.SetUnits(NumOfBikes)

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5

AllWaitMean = []
AllQueueMean = []
AllQueueNum = []
AllServerMean = []


Full_Error = 0
Empty_Error = 0


In [ ]:
class Bike_Sim:

    def Destination(Station_ID):
        Start_Station = Station_ID
        U = np.random.uniform(0, 1)
        D = 1 if U > 1/2 else 0
        Dest_ID = D
        return Dest_ID

    def Departure(Station_ID):
        Stations[Station_ID] -= 1
        Dest_ID = Destination(Station_ID)
        SimFunctions.Schedule(Calendar, "Departure", SimRNG.Lognormal(0.1, np.sqrt(0.066), 1))
        return Dest_ID


    def Arrival(Station_ID, Empty_Error, Full_Error):
        if Station_ID == 0:
            Queue = Queue_1
            Station = Station_1
        else:
            Queue = Queue_2
            Station = Station_2

        SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(MeanTBA, 1))
        Customer = SimClasses.Entity()

        if Station_Capacity[Station_ID] <= Stations[Station_ID]:  # STATION FULL
            Full_Error += 1
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))

        Queue.Add(Customer)
        print(Queue.NumQueue())

        # STATION NOT EMPTY
        if (Station.Busy < Stations[Station_ID]) and (Station.Busy >= 0):
            Station.Seize(1)
            #Dest_ID = Departure(Station_ID)
            SimFunctions.Schedule(Calendar, "EndOfService",
                                SimRNG.Erlang(Phases, MeanST, 2))
        else:
            print(Station.Busy)
            Empty_Error = Empty_Error + 1
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))

        # return Dest_ID


    def Retrial(Dest_ID, Empty_Error, Full_Error):  # retrail function
        if Dest_ID == 0:
            Queue = Queue_1
            Station = Station_1
        else:
            Queue = Queue_2
            Station = Station_2

        Customer = SimClasses.Entity()

        if Station_Capacity[Dest_ID] <= Stations[Dest_ID]:  # STATION FULL
            Full_Error += 1
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))
        else:
            Queue.Add(Customer)
            print(Queue.NumQueue())
            # STATION NOT EMPTY
            if (Station.Busy < Stations[Dest_ID]) and (Station.Busy >= 0):
                Stations[Dest_ID] -= 1
                SimFunctions.Schedule(Calendar, "EndOfService",
                                    SimRNG.Erlang(Phases, MeanST, 2))
            else:
                print(Station.Busy)
                Empty_Error = Empty_Error + 1
                SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(MeanTR, 5))


    def EndOfService(Dest_ID):

        if Dest_ID == 0:
            Queue = Queue_1
            Station = Station_1
        else:
            Queue = Queue_2
            Station = Station_2

        DepartingCustomer = Queue.Remove()
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)

        if Queue.NumQueue() > 0:

            SimFunctions.Schedule(Calendar, "EndOfService",
                                SimRNG.Erlang(Phases, MeanST, 2))
        else:
            Stations[Dest_ID] += 1
            Station.Free(1)


    Station_ID = np.random.random_integers(0, 1)

    for reps in range(0, 1, 1):
        Full_Error = 0
        Empty_Error = 0

        SimFunctions.SimFunctionsInit(
            Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
        SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(MeanTBA, 1))
        SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
        SimFunctions.Schedule(Calendar, "ClearIt", WarmUp)

        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Station_ID = Arrival(Station_ID, Empty_Error, Full_Error)
        elif NextEvent.EventType == "EndOfService":
            EndOfService(Station_ID)
        elif NextEvent.EventType == "ClearIt":
            SimFunctions.ClearStats(TheCTStats, TheDTStats)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime

            if NextEvent.EventType == "Arrival":
                Arrival(Station_ID, Empty_Error, Full_Error)

            elif NextEvent.EventType == "Departure":
                Dest_ID = Departure(Station_ID)

            elif NextEvent.EventType == "Retrial":
                Dest_ID = Retrial(Dest_ID, Empty_Error, Full_Error)

            elif NextEvent.EventType == "EndOfService":
                EndOfService(Dest_ID)

            elif NextEvent.EventType == "ClearIt":
                SimFunctions.ClearStats(TheCTStats, TheDTStats)

        AllWaitMean.append(Wait.Mean())
        AllQueueMean.append(Queue_1.Mean())
        AllQueueNum.append(Queue_1.NumQueue())
        AllServerMean.append(Station_1.Mean())

    q5_output = {"Estimated Expected Average wait:": AllWaitMean, "Estimated Expected Average queue-length:": AllQueueMean,
                "Estimated Expected Queue Number:": AllQueueNum, "Estimated Expected Average utilization:": AllServerMean}
    q5_output
